<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Общее-впечатление" data-toc-modified-id="Общее-впечатление-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><font color="orange">Общее впечатление</font></a></span></li><li><span><a href="#Общее-впечатление-(ревью-2)" data-toc-modified-id="Общее-впечатление-(ревью-2)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span><font color="orange">Общее впечатление (ревью 2)</font></a></span></li></ul></li><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

Импортируем необходимые для работы библиотеки

In [1]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from pymystem3 import Mystem

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score 
from sklearn.metrics import recall_score 
from sklearn.metrics import accuracy_score 
from sklearn.metrics import roc_auc_score 
from sklearn.metrics import roc_curve 
from sklearn.metrics import mean_squared_error
import nltk
from nltk.stem import WordNetLemmatizer 
import nltk
nltk.download('wordnet')

from sklearn.utils import shuffle
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Откроем и взглянем на первые 5 строк датасета

In [2]:
toxic_comments = pd.read_csv('/datasets/toxic_comments.csv')
toxic_comments.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
toxic_comments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
toxic_comments['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

In [5]:
toxic_comments.duplicated().sum()

0

Пропусков в данных нет, дубликатов тоже. Столбцы имеют соответствующий формат. Как мы видим у нас текст на английском языке. То есть при выборе библиотеки надо учесть данный параметр

Прежде чем извлечь признаки из текста, упростим его используя Лемматизацию

In [6]:
%%time

wnl = WordNetLemmatizer()

def clear_text(text):
    text=text.lower()
    pattern = re.sub(r'[^a-zA-Z]', ' ', text)
    clear = pattern.split()
    lemm = []
    for i in range(len(clear)):
        lemm.append(wnl.lemmatize(clear[i]))
    return " ".join(lemm)

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 10.3 µs


In [7]:
toxic_comments['lemm_text'] = toxic_comments['text'].apply(clear_text)
toxic_comments = toxic_comments.drop(['text'], axis=1)
toxic_comments.head()

,Unnamed: 0,toxic,lemm_text
0,0,0,explanation why the edits made under my userna...
1,1,0,d aww he match this background colour i m seem...
2,2,0,hey man i m really not trying to edit war it s...
3,3,0,more i can t make any real suggestion on impro...
4,4,0,you sir are my hero any chance you remember wh...


Определим признаки и целевой признак

In [8]:
target = toxic_comments['toxic']
features = toxic_comments.drop(['toxic'], axis=1)

Разобьем наш датасет сперва на обучающую и валидационную, а потом и на тестовую выборки

In [9]:
features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.4, random_state=12345)
features_valid, features_test, target_valid, target_test = train_test_split(features_valid, target_valid, test_size=0.5, random_state=12345)

Импортирум словарь 'негативных слов' на английском языке и создадим счетчик

In [10]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Преобразуем данные и посмотрим на объемы признаков разных выборок

In [11]:
features_train = count_tf_idf.fit_transform(features_train['lemm_text'])
features_valid = count_tf_idf.transform(features_valid['lemm_text'])
features_test = count_tf_idf.transform(features_test['lemm_text'])
print(features_train.shape)
print(features_valid.shape)
print(features_test.shape)
cv_counts = 3

(95575, 116717)
(31858, 116717)
(31859, 116717)


## Обучение

Расчитаем соотношение позитивных к негативным комментариям

In [12]:
answer_ratio = toxic_comments['toxic'].value_counts()[0] / toxic_comments['toxic'].value_counts()[1]
answer_ratio

8.841344371679229

Обучим 2 модели 

**DecisionTreeClassifier**

In [13]:
best_model = None
best_result = 10000
best_depth = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train, target_train) 
    predicted_valid = model.predict(features_valid) 
    result = mean_squared_error(target_valid, predicted_valid) ** 0.5 
    if result < best_result:
        best_model = model
        best_result = result
        best_depth = depth
print(best_result)

0.2664041753272715


In [14]:
print( 'F1 мера:', f1_score(predicted_valid, target_valid))

F1 мера: 0.5003314917127072


DecisionTree показывает ужасный результат. Думаю подбор параметров нам не поможет

**LogisticRegression**

In [15]:
model = LogisticRegression(random_state=12345, solver = 'liblinear')
model.fit(features_train, target_train)
predicted_valid = model.predict(features_valid)
print('F1 мера:', f1_score(target_valid, predicted_valid))

F1 мера: 0.7262923019709929


Как мы видим резельтат меньше 0.75, что нас не устраивает, но уже близко к требованиям. Подберем параметры и попробуем еще раз обучить модель

Подберем параметры для LogisticRegression

In [16]:
%%time

dict_classes={0:1, 1:answer_ratio}
model = LogisticRegression(class_weight=dict_classes)
train_f1_cros_val = cross_val_score(model, 
                                    features_train, 
                                    target_train, 
                                    cv=cv_counts, 
                                    scoring='f1').mean()
print('F1 с подбором параметров ', train_f1_cros_val)

F1 с подбором параметров  0.7504816736050569
CPU times: user 38.1 s, sys: 53.3 s, total: 1min 31s
Wall time: 1min 31s


LogisticRegression справился с поставленной задачей

Проверим метод на тестовой выборке

In [17]:
model.fit(features_train, target_train)
test_pred = model.predict(features_test)
print(f1_score(target_test, test_pred))

0.7467073339803134


## Выводы

В данном проекте были открыты и изучены данные из датасета toxic_comments.csv. Данные были проверены на дубликаты. Так как текст на английском языке были загружены словари английских негативных слов и создан счетчик. Текст в данных был переведен в нижний регистр, далее была произведена леммитизация. Данные были разделены на обучающую, валидационную и тестовые выборки. Данные были преобразованы. Далее обучили 2 модели. Лучший результат показал метод  LogisticRegression